<center>
<h1><b>Advanced Prompt Engineering with Llama2 and Mistral 7b</b>
</center>

# Objectives

- Present two new techniques for prompt engineering: self-consistency, and tree-of-thought using Llama2 13b and Mistral 7b.


# Setup

Note: This notebook needs to be executed with a GPU runtime since we load and use LLaMA2 & Mistral 7b for inference. Both these models benefit from parallel execution offered by a GPU runtime (code in this notebook runs up to 4 times faster). See screenshots below to create a GPU runtime on a Colab instance.

*Step 1:*

Select the `Runtime` option from the main menu and select the `Change runtime type` from the dropdown.

Select `T4 GPU` from the options presented under Hardware accelerator.

You should now see the T4 GPU mentioned in the runtime logo (screenshot below).

## Installation

In [ ]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --upgrade --no-cache-dir --verbose -q 2>/dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 118.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 184.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 185.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 221.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.28-cp310-cp310-linux_x86_64.whl size=8778602 sha256=482fd63fcf26733d9601380b377d8d4d416922828186434681106e7d6332d8b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-lskzzcjn/wheels/93/6e/a9/478cce089dc2a082bdcffe468a1c65465c91b25d911b30da82
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Succ

In [ ]:
# For downloading the models from HF Hub
!pip install huggingface_hub==0.23.2 -q 2>/dev/null

## Imports

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

# Llama2

### Model

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096 # Context window
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
llama2_prompt_template = """<s>[INST]<<SYS>>
{system_message}
<</SYS>>

{user_message}[/INST]"""

# Problem Statement


In this section, we will explore the process of scaling a baking recipe to accommodate different serving sizes. Baking is both an art and a science, where precise measurements are crucial for achieving the desired outcome. When a recipe is designed for a specific number of servings, adjusting the ingredient quantities for larger or smaller batches can be challenging.

Through iterative prompting techniques, we will investigate how to effectively modify ingredient amounts while ensuring consistency and accuracy in the final product. We will start with a basic recipe and progressively apply different prompting methods to refine our adjustments. By comparing outputs from various approaches, we aim to identify the most effective strategies for scaling recipes, thereby enhancing our understanding of mathematical reasoning in practical cooking scenarios. This exploration will not only highlight the importance of clear calculations but also illustrate how different methodologies can lead to improved recipe outcomes.

## Simple Prompting

In [ ]:
# Simple prompting to adjust recipe ingredients
simple_prompt = """
Adjust the following recipe to serve 10 people.
Original Recipe (Serves 4):
- 2 cups of flour
- 1 cup of sugar
- 0.5 cup of butter
"""

response = lcpp_llm(
    prompt=simple_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False  # Do not return the prompt
)

simple_answer = response["choices"][0]["text"]
print("Simple Answer (Adjusted Recipe):")
print(simple_answer)


Simple Answer (Adjusted Recipe):
- 2 eggs
- 1 teaspoon of vanilla extract
- 1/2 teaspoon of salt
- 1 cup of milk
- 1/4 cup of heavy cream
- 1/2 cup of chocolate chips (semi sweet)

Instructions:

1. Preheat oven to 375 degrees Fahrenheit and grease a 9x13 inch baking dish with butter or cooking spray.
2. In a medium-sized mixing bowl, whisk together flour, sugar, salt, and baking powder until well combined.
3. Add the softened butter to the dry ingredients and mix until the mixture resembles coarse crumbs.
4. Press half of the dough into the prepared baking dish.
5. In a separate bowl, whisk together eggs, vanilla extract, milk, heavy cream, and chocolate chips until well combined.
6. Pour the wet mixture over the dry ingredients in the baking dish.
7. Sprinkle the remaining dough on top of the wet mixture.
8. Bake for 25-30 minutes or until a toothpick inserted into the center comes out clean.
9. Allow the brownies to cool completely before cutting and serving.

Adjusted Recipe (Serve

### Analyzing Simple Prompting Output

The output generated through simple prompting presents a recipe that is clear and straightforward, making it easy for beginners to follow. However, it falls short in several critical areas. Notably, the lack of detailed calculations for adjusting ingredient quantities based on the desired increase in servings may leave learners confused about the scaling process. Additionally, there is no explanation provided on how to adjust these quantities, limiting their understanding of the underlying mathematical principles involved in recipe scaling. Furthermore, the inclusion of unnecessary ingredients and instructional details not relevant to the task contributes to the phenomenon of "hallucination" in the output, which can further mislead learners and detract from the overall quality of the recipe.


## Self-consistency

In self-consistency, we generate multiple answers to the same question and pick the answer that is repeated the most across these occurrences. This is particularly valuable for factual questions.

In [ ]:
# System message to provide context for the LLM
system_message = """
You are a cooking assistant. Adjust recipe ingredient quantities based on desired servings.
"""

# Template for generating multiple answers
answers_template = """
The original recipe serves 4 people:
- 2 cups of flour
- 1 cup of sugar
- 0.5 cup of butter

Adjust the following recipe to serve 10 people. Provide {num_answers} different adjustments.
"""

# Formatting the prompt to generate multiple answers (Self-Consistency)
answers_prompt = llama2_prompt_template.format(
    system_message=system_message,
    user_message=answers_template.format(
        num_answers=5  # Generate 5 distinct answers
    )
)

# Sending the request to generate multiple answers
response = lcpp_llm(
    prompt=answers_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False  # Do not return the prompt
)

# Extracting the generated answers
factual_answers = response["choices"][0]["text"]
print("Generated Answers:")
print(factual_answers)

Llama.generate: prefix-match hit


Generated Answers:
  Sure, I'd be happy to help! Here are five different adjustments to the original recipe to serve 10 people:

Adjustment 1 (Classic Portion):

* Flour: 4 cups (double the amount for 10 servings)
* Sugar: 2 cups (double the amount for 10 servings)
* Butter: 1 cup (keep the same amount, but adjusted for 10 servings)

Adjustment 2 (Large Portion):

* Flour: 5 cups (increase by 1/3 for 10 servings)
* Sugar: 2.5 cups (increase by 1/3 for 10 servings)
* Butter: 1.25 cups (increase by 1/4 for 10 servings)

Adjustment 3 (Small Portion):

* Flour: 3 cups (decrease by 1/3 for 10 servings)
* Sugar: 1.5 cups (decrease by 1/3 for 10 servings)
* Butter: 0.8 cups (decrease by 1/4 for 10 servings)

Adjustment 4 (Low-Fat Version):

* Flour: 3.5 cups (reduce flour by 1/2 cup for a lower carb count)
* Sugar: 1.75 cups (reduce sugar by 0.25 cups for a lower calorie count)
* Butter: 0.6 cups (reduce butter by 0.4 cups for a lower fat count)

Adjustment 5 (Vegan Version):

* Flour: 3.75 c

In [ ]:
# Template for selecting the most frequent answer
consistency_template = """
Here are {num_answers} answers to the recipe adjustment:
Answers:
{answers}

Choose the most frequently suggested quantities for each ingredient.
Final Answer:
"""

# Formatting the prompt to select the most frequent answer
consistency_prompt = llama2_prompt_template.format(
    system_message=system_message,
    user_message=consistency_template.format(
        num_answers=5,  # Number of answers generated previously
        answers=factual_answers  # Pass the generated answers from the previous step
    )
)

# Sending the request to select the most frequent answer
response = lcpp_llm(
    prompt=consistency_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False  # Do not return the prompt
)

# Extracting and printing the final answer
final_answer = response["choices"][0]["text"]
print("Final Answer (Self-Consistency):")
print(final_answer)


Llama.generate: prefix-match hit


Final Answer (Self-Consistency):
  Sure! Based on the five adjustments provided, here are the most frequently suggested quantities for each ingredient for a recipe serving 10 people:

Adjustment 1 (Classic Portion):

* Flour: 4 cups
* Sugar: 2 cups
* Butter: 1 cup

These are the original amounts from the recipe, so there is no need to adjust them for a classic portion.


### Analyzing Self-consistency Prompting Output

The self-consistency prompting approach yields an output that is more tailored to the specific needs of the recipe, addressing some of the issues present in the simple prompting method. This output maintains a degree of consistency in ingredient adjustments and offers multiple perspectives by showcasing different methods for recipe scaling, catering to various preferences. However, it still contains inconsistencies in ingredient amounts for the same serving size, leading to potential confusion regarding measurements like sugar and flour. Additionally, the output lacks clear explanations for why certain quantities were selected over others, leaving users uncertain about which method to trust. This ambiguity could undermine the confidence of learners who are trying to grasp the intricacies of recipe adjustment.

## Tree-of-Thought

Tree-of-thought prompting is a generalization of chain-of-thought prompting where the model is prompted to take multiple reasoning paths. This forces the LLM into a deliberate reasoning mode.

In [ ]:
# Step 1: Tree of Thought Prompting to generate multiple solutions
tot_prompt_template = """
Generate {num_solutions} possible solutions for the following problem:
Problem:
Calculate the ingredient adjustments for the following recipe:
Original Recipe (Serves 4):
- 2 cups of flour
- 1 cup of sugar
- 0.5 cup of butter
Desired Servings: 10 people.

1. Calculate the adjustment factor mathematically: (10 servings / 4 servings).
2. Adjust each ingredient using the adjustment factor.
3. Present the adjusted quantities in bullet points.
"""

tot_prompt = llama2_prompt_template.format(
    system_message=system_message,
    user_message=tot_prompt_template.format(
        num_solutions=5  # Generate 5 distinct solutions
    )
)

# Sending the request to the LLM for generating solutions
response = lcpp_llm(
    prompt=tot_prompt,
    max_tokens=1024,
    temperature=0.7,
    top_p=0.9,
    echo=False  # Do not return the prompt
)

# Extracting and printing generated solutions
tot_solutions = response["choices"][0]["text"]
print("Generated Solutions (Tree of Thought):")
print(tot_solutions)

Llama.generate: prefix-match hit


Generated Solutions (Tree of Thought):
  Sure, I'd be happy to help! Here are five possible solutions for calculating the ingredient adjustments for the recipe:

Solution 1: Adjustment Factor Method

* Calculate the adjustment factor: 10 servings / 4 servings = 2.5
* Adjust each ingredient by multiplying by the adjustment factor:
	+ Flour: 2 cups x 2.5 = 5 cups
	+ Sugar: 1 cup x 2.5 = 2.5 cups
	+ Butter: 0.5 cup x 2.5 = 1.25 cups

Solution 2: Proportional Method

* Divide each ingredient by the number of servings (10) and multiply by the desired number of servings (4):
	+ Flour: 2 cups / 10 = 0.2 cups/serving x 4 = 0.8 cups
	+ Sugar: 1 cup / 10 = 0.1 cups/serving x 4 = 0.4 cups
	+ Butter: 0.5 cup / 10 = 0.05 cups/serving x 4 = 0.2 cups

Solution 3: Scale Method

* Multiply each ingredient by the desired number of servings (4) and divide by the original number of servings (4):
	+ Flour: 2 cups x 4 / 4 = 2 cups x 1 = 2 cups
	+ Sugar: 1 cup x 4 / 4 = 1 cup x 1 = 1 cup
	+ Butter: 0.5 cup x

In [ ]:
# Step 2: Evaluation prompt template
evaluation_template = """
For the following problem: Adjust the recipe to serve 10 people, evaluate each solution based on accuracy and mathematical reasoning:
Solutions:
{solutions}

Present your evaluation of each solution, highlighting strengths and weaknesses.
"""

# Create a new prompt to evaluate the solutions
evaluation_prompt = llama2_prompt_template.format(
    system_message=system_message,
    user_message=evaluation_template.format(
        solutions=tot_solutions
    )
)

# Sending the request to evaluate the solutions
response = lcpp_llm(
    prompt=evaluation_prompt,
    max_tokens=1024,
    temperature=0.7,
    top_p=0.9,
    echo=False  # Do not return the prompt
)

# Extract and print evaluation results
evaluation_results = response["choices"][0]["text"]
print("Evaluation of Solutions:")
print(evaluation_results)

Llama.generate: prefix-match hit


Evaluation of Solutions:
  As a cooking assistant, I would evaluate each solution based on accuracy, mathematical reasoning, and practicality in the kitchen. Here's my evaluation of each solution:

Solution 1: Adjustment Factor Method
Strengths:

* Simple to implement and easy to understand
* Provides a consistent adjustment factor for all ingredients

Weaknesses:

* Does not take into account the specific volume or weight of each ingredient
* May result in over- or under-adjusting certain ingredients, leading to inconsistent flavors or textures

Solution 2: Proportional Method
Strengths:

* Takes into account the specific volume or weight of each ingredient
* Provides a more precise adjustment for each ingredient

Weaknesses:

* Requires more mathematical calculations and can be more time-consuming
* May not be as intuitive or easy to implement as other methods

Solution 3: Scale Method
Strengths:

* Provides a clear and visual representation of the adjustment process
* Easy to unders

In [ ]:
# Step 3: Ranking prompt template
ranking_template = """
For the following problem: Adjust the recipe to serve 10 people, rank the following solutions based on their accuracy and mathematical reasoning:
Evaluations:
{evaluations}

Rank the solutions and explain which one is the best and why.
"""

# Create a new prompt to rank the solutions
ranking_prompt = llama2_prompt_template.format(
    system_message=system_message,
    user_message=ranking_template.format(
        evaluations=evaluation_results
    )
)

# Sending the request to rank the solutions
response = lcpp_llm(
    prompt=ranking_prompt,
    max_tokens=1024,
    temperature=0.7,
    top_p=0.9,
    echo=False  # Do not return the prompt
)

# Extract and print the ranked solutions
ranked_solutions = response["choices"][0]["text"]
print("Ranked Solutions:")
print(ranked_solutions)


Llama.generate: prefix-match hit


Ranked Solutions:
  Based on accuracy and mathematical reasoning, the best solution for adjusting ingredient quantities based on desired servings is the Proportional Method (Solution 2). This method takes into account the specific volume or weight of each ingredient, providing a more precise adjustment for each ingredient. While it may be more time-consuming and require more mathematical calculations, the precision it offers makes it the most reliable method.

The Adjustment Factor Method (Solution 1) is a simple and easy-to-understand method, but it does not take into account the specific volume or weight of each ingredient, which can result in over- or under-adjusting certain ingredients. The Scale Method (Solution 3) provides a clear visual representation of the adjustment process, but it may also result in less precise adjustments.

The Increase Percentage Method (Solution 4) and the Decrease Percentage Method (Solution 5) are easy to understand and implement, but they may not be a

### Analyzing Tree-of-Thought Prompting Output

The tree-of-thought prompting technique provides comprehensive solutions by breaking down each methodical process for scaling recipes, significantly enhancing understanding of various mathematical approaches involved. Unlike the self-consistency prompting approach, which presents conflicting ingredient amounts and lacks clarity on selection rationale, the ToT method systematically organizes information, allowing for the evaluation of each solution based on a clear ranking scheme. By showcasing multiple strategies for achieving the same goal, this output encourages critical thinking and helps in discerning the most effective method for different contexts. Additionally, the structured nature of the ToT output reduces ambiguity, making it easier to grasp complex concepts.

# Mistral 7b

In this section, we will shift our focus to the Mistral family of models, particularly the Mistral 7B, to showcase the effectiveness of various prompting techniques. By leveraging this powerful model, we will explore different methods for crafting prompts that enhance the model's ability to generate accurate and relevant responses. This exploration will highlight how same prompting strategies can influence the Mistral model's performance.

### Model

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf" # the model is in gguf format

In [ ]:
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096 # Context window
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
mistral_prompt_template = """<s>[INST]{prompt}[/INST]"""

## Self-consistency

In self-consistency, we generate multiple answers to the same question and pick the answer that is repeated the most across these occurrences. This is particularly valuable for factual questions.

In [ ]:
answers_template = """
Context:
{context}
===
Using the context above generate {num_answers} distinct answers to the following question:
Question:
{question}.

Arrange your answers in numbered bullet points.
Present only the answers in bullet points.
"""

Here is an extract from the [Tesla 2022 10-K](https://www.sec.gov/Archives/edgar/data/1318605/000095017023001409/tsla-20221231.htm) statement that will be used as context for this demonstration.

In [ ]:
tesla_annual_report_context ="""
In 2022, we recognized total revenues of $81.46 billion, respectively, representing an increase of $27.64 billion, compared to the prior year.
We continue to ramp production, build new manufacturing capacity and expand our operations to enable increased deliveries and deployments of our products and further revenue growth.
"""

In [ ]:
factual_question = "What was the increase in annual revenue in 2022 compared to 2021?"

In [ ]:
answers_prompt = mistral_prompt_template.format(
    prompt=answers_template.format(
        context=tesla_annual_report_context,
        question=factual_question,
        num_answers=3
    )
)

In [ ]:
response = lcpp_llm(
    prompt=answers_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

In [ ]:
factual_answers = response["choices"][0]["text"].strip()

In [ ]:
print(factual_answers)

* The increase in annual revenue from 2021 to 2022 was $27.64 billion.
* In 2022, total revenues were $81.46 billion, which is an increase of $27.64 billion compared to the previous year.
* The company experienced a revenue growth of approximately $27.64 billion between 2021 and 2022.


In [ ]:
consistency_template = """
Here are {num_answers} answers to the question mentioned below:
Question:
{question}
Answers:
{answers}

Observe the answers mentioned above and choose the answer that occurs most.
Present only the most frequent solution in the following format.
Final Answer:
"""

In [ ]:
consistency_prompt = mistral_prompt_template.format(
    prompt=consistency_template.format(
        num_answers=3,
        question=factual_question,
        answers=factual_answers
    )
)

In [ ]:
response = lcpp_llm(
    prompt=consistency_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
print(response["choices"][0]["text"].strip())

Final Answer: The increase in annual revenue from 2021 to 2022 was $27.64 billion.


## Tree-of-Thought

Tree-of-thought prompting is a generalization of chain-of-thought prompting where the model is prompted to take multiple reasoning paths. This forces the LLM into a deliberate reasoning mode.

In [ ]:
solutions_template = """
Generate {num_solutions} distinct solutions for the following problem:
Problem:
{problem}.
--

Consider the following factors in coming up with your solutions.
Factors:
{factors}

Present the {num_solutions} solutions in numbered bullet points. Present only the solutions.
"""

In [ ]:
climate_problem = "Reduce the impact of climate change on the occurrence of extreme events in the Earth's atmosphere."

climate_factors = """
1. Renewable Energy Transition
2. Reforestation
3. Sustainable Agricultural Practises
4. Carbon capture and storage
5. Climate-resilient infrastructure
6. Circular economy practises
"""

In [ ]:
solutions_prompt = mistral_prompt_template.format(
    prompt=solutions_template.format(
        num_solutions=3,
        problem=climate_problem,
        factors=climate_factors
    )
)

In [ ]:
response = lcpp_llm(
    prompt=solutions_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
climate_solutions = response["choices"][0]["text"].strip()

In [ ]:
print(climate_solutions)

Solution 1:
- Accelerate the transition to renewable energy sources by investing in solar, wind, hydroelectric, and other clean energy technologies.
- Encourage governments and businesses to set ambitious targets for reducing greenhouse gas emissions.
- Implement policies that incentivize individuals and organizations to adopt renewable energy solutions.

Solution 2:
- Increase global reforestation efforts by planting new trees, protecting existing forests, and promoting sustainable forest management practices.
- Encourage the use of agroforestry systems, which combine agriculture with tree cultivation, to help sequester carbon and reduce deforestation.
- Support research into innovative approaches for afforestation (planting new forests in previously treeless areas) and reforestation (reestablishing forests on degraded or deforested land).

Solution 3:
- Implement climate-resilient infrastructure projects, such as green roofs, rainwater harvesting systems, and permeable pavements.
- E

In [ ]:
evaluation_template = """
For the following problem: {problem}, evaluate each solution in the following proposed solutions: \n{solutions}\n.
Analyze pros, cons, feasibility, and probability of success for each solution.
Present your evaluations of each solutions.
"""

In [ ]:
evaluations_prompt = mistral_prompt_template.format(
    prompt=evaluation_template.format(
        problem=climate_problem,
        solutions=climate_solutions
    )
)

In [ ]:
response = lcpp_llm(
    prompt=evaluations_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
climate_proposal_evaluations = response["choices"][0]["text"].strip()

In [ ]:
print(climate_proposal_evaluations)

Solution 1: Transitioning to Renewable Energy Sources
Pros:
- Reduces greenhouse gas emissions from the power sector, which is a major contributor to climate change.
- Creates jobs and stimulates economic growth in the renewable energy industry.
- Decreases dependence on fossil fuels, reducing vulnerability to price volatility and supply disruptions.
Cons:
- High upfront costs for infrastructure development and implementation of renewable energy technologies.
- Intermittency of some renewable energy sources (e.g., solar and wind) may require backup power or energy storage solutions.
- Political challenges in implementing policies that incentivize the transition to renewable energy, particularly in countries with strong fossil fuel industries.
Feasibility: Renewable energy technologies have become increasingly cost-competitive with traditional fossil fuels, making them a viable alternative for many applications. However, significant investment is still required to build out infrastructu

In [ ]:
ranking_template = """
For the following problem: {problem}, rank the solutions presented in the following evaluations: \n{evaluations}\n.
Pick most promising solution and present implementation strategies and methods to handle potential obstacles for this solution.
"""

In [ ]:
ranking_prompt = mistral_prompt_template.format(
    prompt=ranking_template.format(
        problem=climate_problem,
        evaluations=climate_proposal_evaluations
    )
)

In [ ]:
response = lcpp_llm(
    prompt=ranking_prompt,
    max_tokens=1024,
    temperature=0,
    top_p=0.95,
    repeat_penalty=1.2,
    echo=False # do not return the prompt
)

Llama.generate: prefix-match hit


In [ ]:
climate_proposal_rankings = response["choices"][0]["text"].strip()

In [ ]:
print(climate_proposal_rankings)

Based on the evaluations provided, all three solutions - transitioning to renewable energy sources, reforestation and forest management, and climate-resilient infrastructure and circular economy practices - have their merits in reducing the impact of climate change on extreme events in the Earth's atmosphere. However, considering the current state of technology, economics, and political will, I would rank Solution 1: Transitioning to Renewable Energy Sources as the most promising solution for several reasons.

Firstly, renewable energy sources have become increasingly cost-competitive with traditional fossil fuels in many applications, making them a viable alternative for power generation. This trend is expected to continue as technology advances and economies of scale are achieved. Moreover, transitioning to renewable energy reduces greenhouse gas emissions from the power sector, which is one of the largest contributors to climate change. Additionally, it creates jobs and stimulates e